本篇note主要针对于seed数据集以及自采数据集,为保证单个使用的效果,各模块均有重复

In [1]:
#@title pip安装必须的包
!pip install keras-tuner
!pip install scikit-learn
!pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 10.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#@title 架构 tcn模块以及mlp
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, Dense, Dropout, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

input_shape = (4, 512)
num_classes = 2

def tcn_block(x, dilation_rate, nb_filters, kernel_size, padding, use_bias, dropout_rate):
    x = Conv1D(filters=nb_filters, kernel_size=kernel_size, dilation_rate=dilation_rate, padding=padding, use_bias=use_bias, kernel_regularizer=l2(0.001))(x)
    x = Dropout(dropout_rate)(x)
    return x

def tcn_model(input_shape, num_classes):
    input_layer = Input(shape=input_shape)
    x = tf.transpose(input_layer, perm = [0,2,1])
    print(x.shape)
    x = tcn_block(x, dilation_rate=1, nb_filters=64, kernel_size=3, padding='causal', use_bias=True, dropout_rate=0.2)
    x = tcn_block(x, dilation_rate=2, nb_filters=64, kernel_size=3, padding='causal', use_bias=True, dropout_rate=0.2)
    x = tcn_block(x, dilation_rate=3, nb_filters=64, kernel_size=3, padding='causal', use_bias=True, dropout_rate=0.2)
    # x = tcn_block(x, dilation_rate=8, nb_filters=64, kernel_size=3, padding='causal', use_bias=True, dropout_rate=0.2)
    return Model(inputs=input_layer,outputs = x[:,:,-1])
  
def mlp(input_dim,num_classes):
    model = Sequential()
    model.add(Dense(64, activation='relu', input_dim=input_dim))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    return model

    
ml = tf.random.normal([2,4,512])
model = tcn_model(input_shape,num_classes)
out = model(ml)
newout = LSTM(128)(ml)
out.shape,newout.shape

(None, 512, 4)


(TensorShape([2, 512]), TensorShape([2, 128]))

In [ ]:
#@title SEED 训练SVM 支持向量机
# from EEGModels import EEGNet
from sklearn.model_selection import StratifiedKFold
from keras.utils import to_categorical
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

num_classes = 3
img_rows, img_cols, num_chan = 8, 9, 4

img_size = (img_rows, img_cols, num_chan)

falx = np.load('/content/drive/MyDrive/t4x_89.npy')
y = np.load('/content/drive/MyDrive/t4y_89.npy')
one_y_1 = np.array([y[:1692]] * 3).reshape((-1,))
all_acc = []
for nb in range(1):
    one_falx_1 = falx[nb * 3:nb * 3 + 3]
    one_falx_1 = one_falx_1.reshape((-1, 4, img_rows, img_cols, 5))
    one_y = one_y_1
    one_falx = one_falx_1[:,:,:,:,1:5]
    print(one_y.shape)
    print(one_falx.shape)
    seed = 7
    np.random.seed(seed)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    cvscores = []
    x = one_falx.reshape((one_falx.shape[0],-1))
    y = one_y
    # create model
    for train, test in kfold.split(x, y):
      x_train = x[train]
      y_train = y[train]
      x_test = x[test]
      y_test = y[test]
      clf = svm.SVC()
      clf.fit(x_train, y_train)
      accuracy = clf.score(x_test, y_test)
      print("Model accuracy:", accuracy)
      print("%.2f%%" % (accuracy * 100)) # Accuracy
      all_acc.append(accuracy * 100)

# print("all acc: {}".format(all_acc))
print("mean acc: {}".format(np.mean(all_acc)))
print("std acc: {}".format(np.std(all_acc)))

  

(5076,)
(5076, 4, 8, 9, 4)
Model accuracy: 0.7854330708661418
78.54%
Model accuracy: 0.7901477832512315
79.01%
Model accuracy: 0.7901477832512315
79.01%
Model accuracy: 0.7793103448275862
77.93%
Model accuracy: 0.7694581280788177
76.95%
mean acc: 78.28994220550018
std acc: 0.7810847783163624


In [ ]:
#@title SEED 训练LR 逻辑回归
# from EEGModels import EEGNet
from sklearn.model_selection import StratifiedKFold
from keras.utils import to_categorical
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

num_classes = 3
img_rows, img_cols, num_chan = 8, 9, 4

img_size = (img_rows, img_cols, num_chan)

falx = np.load('/content/drive/MyDrive/t4x_89.npy')
y = np.load('/content/drive/MyDrive/t4y_89.npy')
one_y_1 = np.array([y[:1692]] * 3).reshape((-1,))
all_acc = []
for nb in range(1):
    one_falx_1 = falx[nb * 3:nb * 3 + 3]
    one_falx_1 = one_falx_1.reshape((-1, 4, img_rows, img_cols, 5))
    one_y = one_y_1
    one_falx = one_falx_1[:,:,:,:,1:5]
    print(one_y.shape)
    print(one_falx.shape)
    seed = 7
    np.random.seed(seed)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    cvscores = []
    x = one_falx.reshape((one_falx.shape[0],-1))
    y = one_y
    # create model
    for train, test in kfold.split(x, y):
      x_train = x[train]
      y_train = y[train]
      x_test = x[test]
      y_test = y[test]
      clf = LogisticRegression(max_iter=1000)
      clf.fit(x_train, y_train)
      accuracy = clf.score(x_test, y_test)
      print("Model accuracy:", accuracy)
      print("%.2f%%" % (accuracy * 100)) # Accuracy
      all_acc.append(accuracy * 100)

# print("all acc: {}".format(all_acc))
print("mean acc: {}".format(np.mean(all_acc)))
print("std acc: {}".format(np.std(all_acc)))


(5076,)
(5076, 4, 8, 9, 4)


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Model accuracy: 0.765748031496063
76.57%


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Model accuracy: 0.7733990147783252
77.34%


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Model accuracy: 0.7773399014778325
77.73%


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Model accuracy: 0.7399014778325124
73.99%
Model accuracy: 0.7586206896551724
75.86%
mean acc: 76.30018230479811
std acc: 1.3221528116355041


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
#@title SEED 训练DT 决策树
# from EEGModels import EEGNet
from sklearn.model_selection import StratifiedKFold
from keras.utils import to_categorical
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

num_classes = 3
img_rows, img_cols, num_chan = 8, 9, 4

img_size = (img_rows, img_cols, num_chan)

falx = np.load('/content/drive/MyDrive/t4x_89.npy')
y = np.load('/content/drive/MyDrive/t4y_89.npy')
one_y_1 = np.array([y[:1692]] * 3).reshape((-1,))
all_acc = []
for nb in range(1):
    one_falx_1 = falx[nb * 3:nb * 3 + 3]
    one_falx_1 = one_falx_1.reshape((-1, 4, img_rows, img_cols, 5))
    one_y = one_y_1
    one_falx = one_falx_1[:,:,:,:,1:5]
    print(one_y.shape)
    print(one_falx.shape)
    seed = 7
    np.random.seed(seed)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    cvscores = []
    x = one_falx.reshape((one_falx.shape[0],-1))
    y = one_y
    # create model
    for train, test in kfold.split(x, y):
      x_train = x[train]
      y_train = y[train]
      x_test = x[test]
      y_test = y[test]
      clf = DecisionTreeClassifier()
      clf.fit(x_train, y_train)
      accuracy = clf.score(x_test, y_test)
      print("Model accuracy:", accuracy)
      print("%.2f%%" % (accuracy * 100)) # Accuracy
      all_acc.append(accuracy * 100)

# print("all acc: {}".format(all_acc))
print("mean acc: {}".format(np.mean(all_acc)))
print("std acc: {}".format(np.std(all_acc)))


(5076,)
(5076, 4, 8, 9, 4)
Model accuracy: 0.6840551181102362
68.41%
Model accuracy: 0.7408866995073892
74.09%
Model accuracy: 0.7093596059113301
70.94%
Model accuracy: 0.6926108374384237
69.26%
Model accuracy: 0.7467980295566502
74.68%
mean acc: 71.47420581048058
std acc: 2.5185621238399523


In [ ]:
#@title SEED 训练RF 随机森林
# from EEGModels import EEGNet
from sklearn.model_selection import StratifiedKFold
from keras.utils import to_categorical
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

num_classes = 3
img_rows, img_cols, num_chan = 8, 9, 4

img_size = (img_rows, img_cols, num_chan)

falx = np.load('/content/drive/MyDrive/t4x_89.npy')
y = np.load('/content/drive/MyDrive/t4y_89.npy')
one_y_1 = np.array([y[:1692]] * 3).reshape((-1,))
all_acc = []
for nb in range(1):
    one_falx_1 = falx[nb * 3:nb * 3 + 3]
    one_falx_1 = one_falx_1.reshape((-1, 4, img_rows, img_cols, 5))
    one_y = one_y_1
    one_falx = one_falx_1[:,:,:,:,1:5]
    print(one_y.shape)
    print(one_falx.shape)
    seed = 7
    np.random.seed(seed)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    cvscores = []
    x = one_falx.reshape((one_falx.shape[0],-1))
    y = one_y
    # create model
    for train, test in kfold.split(x, y):
      x_train = x[train]
      y_train = y[train]
      x_test = x[test]
      y_test = y[test]
      clf = RandomForestClassifier(n_estimators=10)
      clf.fit(x_train, y_train)
      accuracy = clf.score(x_test, y_test)
      print("Model accuracy:", accuracy)
      print("%.2f%%" % (accuracy * 100)) # Accuracy
      all_acc.append(accuracy * 100)

# print("all acc: {}".format(all_acc))
print("mean acc: {}".format(np.mean(all_acc)))
print("std acc: {}".format(np.std(all_acc)))


(5076,)
(5076, 4, 8, 9, 4)
Model accuracy: 0.7755905511811023
77.56%
Model accuracy: 0.7793103448275862
77.93%
Model accuracy: 0.7714285714285715
77.14%
Model accuracy: 0.7704433497536946
77.04%
Model accuracy: 0.7517241379310344
75.17%
mean acc: 76.96993910243978
std acc: 0.9526063869516189


In [ ]:
#@title SEED 训练MLP 多层感知机
import tensorflow as tf
import numpy as np
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout,Bidirectional
from keras.layers import Flatten, Dense, Concatenate, Reshape, LSTM
from keras.models import Sequential, Model

import keras
from keras import backend as K
import time
from sklearn.model_selection import StratifiedKFold


num_classes = 3
batch_size = 128
img_rows, img_cols, num_chan = 8, 9, 4

img_size = (img_rows, img_cols, num_chan)


falx = np.load('/content/drive/MyDrive/t4x_89.npy')
y = np.load('/content/drive/MyDrive/t4y_89.npy')
# one_y = np.array([y[:1697]] * 3).reshape((-1,))
# one_y = to_categorical(one_y, num_classes)
one_y_1 = np.array([y[:1692]] * 3).reshape((-1,))
one_y_1 = to_categorical(one_y_1, num_classes)

acc_list = []
std_list = []
all_acc = []
for nb in range(1):
    K.clear_session()
    start = time.time()
    one_falx_1 = falx[nb * 3:nb * 3 + 3]
    one_falx_1 = one_falx_1.reshape((-1, 4, img_rows, img_cols, 5))

    one_y = one_y_1
    one_falx = one_falx_1[:,:,:,:,1:5]
    one_falx = one_falx.transpose((0,2,3,1,4))
    one_falx = one_falx.reshape((one_falx.shape[0],-1))
    print(one_y.shape)
    print(one_falx.shape)
    # x_train, x_test, y_train, y_test = train_test_split(one_falx, one_y, test_size=0.25)
    seed = 7
    np.random.seed(seed)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    cvscores = []

    # create model
    for train, test in kfold.split(one_falx, one_y.argmax(1)):
        model = mlp(one_falx.shape[-1],3)
        model.compile(loss=keras.losses.categorical_crossentropy,
                      optimizer=keras.optimizers.Adam(),
                      metrics=['accuracy'])
        # Fit the model

        x_train = one_falx[train]
        print('x train ',x_train.shape)
        y_train = one_y[train]
        model.fit(x_train, y_train, epochs=100, batch_size=128, verbose=0)
        # evaluate the model
        x_test = one_falx[test]
        y_test = one_y[test]
        scores = model.evaluate(x_test, y_test, verbose=0)

        print("%.2f%%" % (scores[1] * 100)) # Accuracy
        all_acc.append(scores[1] * 100)

    # print("all acc: {}".format(all_acc))
    print("mean acc: {}".format(np.mean(all_acc)))
    print("std acc: {}".format(np.std(all_acc)))
    acc_list.append(np.mean(all_acc))
    std_list.append(np.std(all_acc))
    print("进度： {}".format(nb))
    all_acc = []
    end = time.time()
    print("%.2f" % (end - start))   # run time
print('Acc_all: {}'.format(acc_list))
print('Std_all: {}'.format(std_list))
print("Acc_mean: {}".format(np.mean(acc_list)))
print("Std_all: {}".format(np.std(std_list)))


(5076, 3)
(5076, 1152)
x train  (4060, 1152)
80.31%
x train  (4061, 1152)
70.54%
x train  (4061, 1152)
81.97%
x train  (4061, 1152)
77.93%
x train  (4061, 1152)
80.39%
mean acc: 78.23047876358032
std acc: 4.055069587504695
进度： 0
64.56
Acc_all: [78.23047876358032]
Std_all: [4.055069587504695]
Acc_mean: 78.23047876358032
Std_all: 0.0


In [ ]:
#@title SEED 训练CNN 使用CNN进行消融
import tensorflow as tf
import numpy as np
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout,Bidirectional
from keras.layers import Flatten, Dense, Concatenate, Reshape, LSTM
from keras.models import Sequential, Model

import keras
from keras import backend as K
import time
from sklearn.model_selection import StratifiedKFold


num_classes = 3
batch_size = 128
img_rows, img_cols, num_chan = 8, 9, 4

img_size = (img_rows, img_cols, num_chan)


falx = np.load('/content/drive/MyDrive/t4x_89.npy')
y = np.load('/content/drive/MyDrive/t4y_89.npy')
# one_y = np.array([y[:1697]] * 3).reshape((-1,))
# one_y = to_categorical(one_y, num_classes)
one_y_1 = np.array([y[:1692]] * 3).reshape((-1,))
one_y_1 = to_categorical(one_y_1, num_classes)

acc_list = []
std_list = []
all_acc = []
for nb in range(1):
    K.clear_session()
    start = time.time()
    one_falx_1 = falx[nb * 3:nb * 3 + 3]
    one_falx_1 = one_falx_1.reshape((-1, 4, img_rows, img_cols, 5))

    one_y = one_y_1
    one_falx = one_falx_1[:,:,:,:,1:5]
    one_falx = one_falx.transpose((0,2,3,1,4))
    one_falx = one_falx.reshape(-1,8,9,16)
    print(one_y.shape)
    print(one_falx.shape)
    # x_train, x_test, y_train, y_test = train_test_split(one_falx, one_y, test_size=0.25)
    seed = 7
    np.random.seed(seed)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    cvscores = []

    # create model
    for train, test in kfold.split(one_falx, one_y.argmax(1)):
        img_size = (img_rows, img_cols, num_chan*4)

        # model.summary()
        def create_base_network(input_dim):
          seq = Sequential()
          seq.add(Conv2D(64, 5, activation='relu', padding='same', name='conv1', input_shape=input_dim))
          seq.add(Conv2D(128, 4, activation='relu', padding='same', name='conv2'))
          seq.add(Conv2D(256, 4, activation='relu', padding='same', name='conv3'))
          seq.add(Conv2D(64, 1, activation='relu', padding='same', name='conv4'))
          seq.add(MaxPooling2D(2, 2, name='pool1'))
          seq.add(Flatten(name='fla1'))
          seq.add(Dense(512, activation='relu', name='dense1'))
          return seq
        base_network = create_base_network(img_size)
        
        input_4 = Input(shape=img_size)
        out_all = base_network(input_4)
        out_layer = Dense(3, activation='softmax', name='out')(out_all)
        model = Model(inputs = input_4, outputs = out_layer)
        
        model.compile(loss=keras.losses.categorical_crossentropy,
                      optimizer=keras.optimizers.Adam(),
                      metrics=['accuracy'])


        x_train = one_falx[train]
        print('x train ',x_train.shape)
        y_train = one_y[train]

        model.fit(x_train, y_train, epochs=100, batch_size=128, verbose=0)
        # evaluate the model
        x_test = one_falx[test]
        y_test = one_y[test]
        scores = model.evaluate(x_test, y_test, verbose=0)

        print("%.2f%%" % (scores[1] * 100)) # Accuracy
        all_acc.append(scores[1] * 100)

    # print("all acc: {}".format(all_acc))
    print("mean acc: {}".format(np.mean(all_acc)))
    print("std acc: {}".format(np.std(all_acc)))
    acc_list.append(np.mean(all_acc))
    std_list.append(np.std(all_acc))
    print("进度： {}".format(nb))
    all_acc = []
    end = time.time()
    print("%.2f" % (end - start))   # run time
print('Acc_all: {}'.format(acc_list))
print('Std_all: {}'.format(std_list))
print("Acc_mean: {}".format(np.mean(acc_list)))
print("Std_all: {}".format(np.std(std_list)))


In [ ]:
#@title SEED 训练LSTM 使用LSTM进行消融
import tensorflow as tf
import numpy as np
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from keras.layers import Input, Conv2D, MaxPooling2D, Dropout,Bidirectional
from keras.layers import Flatten, Dense, Concatenate, Reshape, LSTM
from keras.models import Sequential, Model

import keras
from keras import backend as K
import time
from sklearn.model_selection import StratifiedKFold


num_classes = 3
batch_size = 128
img_rows, img_cols, num_chan = 8, 9, 4

img_size = (img_rows, img_cols, num_chan)


falx = np.load('/content/drive/MyDrive/t4x_89.npy')
y = np.load('/content/drive/MyDrive/t4y_89.npy')
# one_y = np.array([y[:1697]] * 3).reshape((-1,))
# one_y = to_categorical(one_y, num_classes)
one_y_1 = np.array([y[:1692]] * 3).reshape((-1,))
one_y_1 = to_categorical(one_y_1, num_classes)

acc_list = []
std_list = []
all_acc = []
for nb in range(1):
    K.clear_session()
    start = time.time()
    one_falx_1 = falx[nb * 3:nb * 3 + 3]
    one_falx_1 = one_falx_1.reshape((-1, 4, img_rows, img_cols, 5))

    one_y = one_y_1
    one_falx = one_falx_1[:,:,:,:,1:5]
   
    one_falx = one_falx.reshape(-1,4,288)
    print(one_y.shape)
    print(one_falx.shape)
    # x_train, x_test, y_train, y_test = train_test_split(one_falx, one_y, test_size=0.25)
    seed = 7
    np.random.seed(seed)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    cvscores = []

    # create model
    for train, test in kfold.split(one_falx, one_y.argmax(1)):
        img_size = (1,288)

        input_1 = Input(shape=img_size)
        input_2 = Input(shape=img_size)
        input_3 = Input(shape=img_size)
        input_4 = Input(shape=img_size)
        out_all = Concatenate(axis=1)([input_1, input_2, input_3, input_4])
        lstm = LSTM(128)(out_all)
        out_layer = Dense(3, activation='softmax', name='out')(lstm)
        model = Model(inputs = [input_1, input_2, input_3, input_4], outputs = out_layer)

        # Compile model
        model.compile(loss=keras.losses.categorical_crossentropy,
                      optimizer=keras.optimizers.Adam(),
                      metrics=['accuracy'])
        # Fit the model

        x_train = one_falx[train]
        print('x train ',x_train.shape)
        y_train = one_y[train]

        model.fit([x_train[:, 0].reshape(-1,1,288), x_train[:, 1].reshape(-1,1,288), x_train[:, 2].reshape(-1,1,288), x_train[:, 3].reshape(-1,1,288)],y_train, epochs=100, batch_size=128, verbose=0)
        # evaluate the model
        x_test = one_falx[test]
        y_test = one_y[test]
        scores = model.evaluate([x_test[:, 0].reshape(-1,1,288), x_test[:, 1].reshape(-1,1,288), x_test[:, 2].reshape(-1,1,288), x_test[:, 3].reshape(-1,1,288)], y_test, verbose=0)

        print("%.2f%%" % (scores[1] * 100)) # Accuracy
        all_acc.append(scores[1] * 100)

    # print("all acc: {}".format(all_acc))
    print("mean acc: {}".format(np.mean(all_acc)))
    print("std acc: {}".format(np.std(all_acc)))
    acc_list.append(np.mean(all_acc))
    std_list.append(np.std(all_acc))
    print("进度： {}".format(nb))
    all_acc = []
    end = time.time()
    print("%.2f" % (end - start))   # run time
print('Acc_all: {}'.format(acc_list))
print('Std_all: {}'.format(std_list))
print("Acc_mean: {}".format(np.mean(acc_list)))
print("Std_all: {}".format(np.std(std_list)))


(5076, 3)
(5076, 4, 288)
x train  (4060, 4, 288)
85.73%
x train  (4061, 4, 288)
83.74%
x train  (4061, 4, 288)
83.45%
x train  (4061, 4, 288)
85.02%
x train  (4061, 4, 288)
82.66%
mean acc: 84.12103772163391
std acc: 1.1071732742537355
进度： 0
74.40
Acc_all: [84.12103772163391]
Std_all: [1.1071732742537355]
Acc_mean: 84.12103772163391
Std_all: 0.0


In [ ]:
#@title 架构 CNN-TCN
def CNN_TCN():
    def create_base_network(input_dim):
        seq = Sequential()
        seq.add(Conv2D(64, 5, activation='relu', padding='same', name='conv1', input_shape=input_dim))
        seq.add(Conv2D(128, 4, activation='relu', padding='same', name='conv2'))
        seq.add(Conv2D(256, 4, activation='relu', padding='same', name='conv3'))
        seq.add(Conv2D(64, 1, activation='relu', padding='same', name='conv4'))
        seq.add(MaxPooling2D(2, 2, name='pool1'))
        seq.add(Flatten(name='fla1'))
        seq.add(Dense(512, activation='relu', name='dense1'))
        seq.add(Reshape((1, 512), name='reshape'))
        return seq
    base_network = create_base_network(img_size)
    input_1 = Input(shape=img_size)
    input_2 = Input(shape=img_size)
    input_3 = Input(shape=img_size)
    input_4 = Input(shape=img_size)
    out_all = Concatenate(axis=1)([base_network(input_1), base_network(input_2), base_network(input_3), base_network(input_4)])
    tcn_layer = tcn_model((4,512),3)(out_all)
    # fat = Flatten(tcn_layer)
    out_layer = Dense(3, activation='softmax', name='out')(tcn_layer)
    return Model(inputs = [input_1, input_2, input_3, input_4], outputs = out_layer)

In [ ]:
#@title SEED 训练CNN-TCN模型
import numpy as np
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from keras.layers import Input, Conv2D, MaxPooling2D, Dropout,Bidirectional
from keras.layers import Flatten, Dense, Concatenate, Reshape, LSTM
from keras.models import Sequential, Model

import keras
from keras import backend as K
import time
from sklearn.model_selection import StratifiedKFold


num_classes = 3
batch_size = 128
img_rows, img_cols, num_chan = 8, 9, 4

img_size = (img_rows, img_cols, num_chan)


falx = np.load('/content/drive/MyDrive/t4x_89.npy')
y = np.load('/content/drive/MyDrive/t4y_89.npy')
# one_y = np.array([y[:1697]] * 3).reshape((-1,))
# one_y = to_categorical(one_y, num_classes)
one_y_1 = np.array([y[:1692]] * 3).reshape((-1,))
one_y_1 = to_categorical(one_y_1, num_classes)

acc_list = []
std_list = []
all_acc = []
for nb in range(1):
    K.clear_session()
    start = time.time()
    one_falx_1 = falx[nb * 3:nb * 3 + 3]
    one_falx_1 = one_falx_1.reshape((-1, 4, img_rows, img_cols, 5))

    one_y = one_y_1
    one_falx = one_falx_1[:,:,:,:,1:5]

    print(one_y.shape)
    print(one_falx.shape)
    # x_train, x_test, y_train, y_test = train_test_split(one_falx, one_y, test_size=0.25)
    seed = 7
    np.random.seed(seed)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    cvscores = []

    # create model
    for train, test in kfold.split(one_falx, one_y.argmax(1)):
        img_size = (img_rows, img_cols, num_chan)

        # model.summary()
        def create_base_network(input_dim):
          seq = Sequential()
          seq.add(Conv2D(64, 5, activation='relu', padding='same', name='conv1', input_shape=input_dim))
          seq.add(Conv2D(128, 4, activation='relu', padding='same', name='conv2'))
          seq.add(Conv2D(256, 4, activation='relu', padding='same', name='conv3'))
          seq.add(Conv2D(64, 1, activation='relu', padding='same', name='conv4'))
          seq.add(MaxPooling2D(2, 2, name='pool1'))
          seq.add(Flatten(name='fla1'))
          seq.add(Dense(512, activation='relu', name='dense1'))
          seq.add(Reshape((1, 512), name='reshape'))
          return seq
        base_network = create_base_network(img_size)
        input_1 = Input(shape=img_size)
        input_2 = Input(shape=img_size)
        input_3 = Input(shape=img_size)
        input_4 = Input(shape=img_size)
        out_all = Concatenate(axis=1)([base_network(input_1), base_network(input_2), base_network(input_3), base_network(input_4)])
        tcn_layer = tcn_model((4,512),3)(out_all)
    # fat = Flatten(tcn_layer)
        out_layer = Dense(3, activation='softmax', name='out')(tcn_layer)
        model = Model(inputs = [input_1, input_2, input_3, input_4], outputs = out_layer)
        # Compile model
        model.compile(loss=keras.losses.categorical_crossentropy,
                      optimizer=keras.optimizers.Adam(),
                      metrics=['accuracy'])
        # Fit the model
        x_train = one_falx[train]
        y_train = one_y[train]

        model.fit([x_train[:, 0], x_train[:, 1], x_train[:, 2], x_train[:, 3]], y_train, epochs=100, batch_size=128, verbose=0)
        # evaluate the model
        x_test = one_falx[test]
        y_test = one_y[test]
        scores = model.evaluate([x_test[:, 0], x_test[:, 1], x_test[:, 2], x_test[:, 3]], y_test, verbose=0)

        print("%.2f%%" % (scores[1] * 100)) # Accuracy
        all_acc.append(scores[1] * 100)

    # print("all acc: {}".format(all_acc))
    print("mean acc: {}".format(np.mean(all_acc)))
    print("std acc: {}".format(np.std(all_acc)))
    acc_list.append(np.mean(all_acc))
    std_list.append(np.std(all_acc))
    print("进度： {}".format(nb))
    all_acc = []
    end = time.time()
    print("%.2f" % (end - start))   # run time
print('Acc_all: {}'.format(acc_list))
print('Std_all: {}'.format(std_list))
print("Acc_mean: {}".format(np.mean(acc_list)))
print("Std_all: {}".format(np.std(std_list)))


(5076, 3)
(5076, 4, 8, 9, 4)
(None, 512, 4)
91.24%
(None, 512, 4)
91.03%
(None, 512, 4)
92.41%
(None, 512, 4)
87.29%
(None, 512, 4)
93.30%
mean acc: 91.05591297149658
std acc: 2.053955779302763
进度： 0
1023.80
Acc_all: [91.05591297149658]
Std_all: [2.053955779302763]
Acc_mean: 91.05591297149658
Std_all: 0.0


In [ ]:
#@title !自采数据集 CRNN训练自己的数据
import numpy as np
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from keras.layers import Input, Conv2D, MaxPooling2D, Dropout,Bidirectional
from keras.layers import Flatten, Dense, Concatenate, Reshape, LSTM
from keras.models import Sequential, Model

import keras
from keras import backend as K
import time
from sklearn.model_selection import StratifiedKFold


num_classes = 2
batch_size = 6
img_rows, img_cols, num_chan = 5, 5, 5

img_size = (img_rows, img_cols, num_chan)


x = np.load('/content/drive/MyDrive/X52.npy')
y = np.load('/content/drive/MyDrive/EEG_Y_16ch.npy')
x = x.transpose((0,1,4,2,3))
y = to_categorical(y, num_classes)

acc_list = []
std_list = []
all_acc = []

start = time.time()
# x_train, x_test, y_train, y_test = train_test_split(one_falx, one_y, test_size=0.25)
seed = 7
np.random.seed(seed)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
cvscores = []

# create model
for train, test in kfold.split(x, y.argmax(1)):
    img_size = (img_rows, img_cols, num_chan)

    # model.summary()
    def create_base_network(input_dim):
      seq = Sequential()
      seq.add(Conv2D(64, 3, activation='relu', padding='same', name='conv1', input_shape=input_dim))
      seq.add(Conv2D(128, 2, activation='relu', padding='same', name='conv2'))
      seq.add(Conv2D(256, 2, activation='relu', padding='same', name='conv3'))
      seq.add(Conv2D(64, 1, activation='relu', padding='same', name='conv4'))
      seq.add(MaxPooling2D(2, 2, name='pool1'))
      seq.add(Flatten(name='fla1'))
      seq.add(Dense(512, activation='relu', name='dense1'))
      seq.add(Reshape((1, 512), name='reshape'))
      return seq
    base_network = create_base_network(img_size)
    input_list = []
    for i in range(74):
      m = Input(shape=img_size)
      input_list.append(m)
    base_list = []
    for i in range(74):
      m = base_network(input_list[i])
      base_list.append(m)
    out_all = Concatenate(axis=1)(base_list)
    newout = LSTM(128)(out_all)
    out_layer = Dense(2, activation='softmax', name='out')(newout)
    model = Model(inputs = input_list, outputs = out_layer)
    # Compile model
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adam(),
                  metrics=['accuracy'])
    # Fit the model
    x_train = x[train]
    y_train = y[train]
    train_list = [x_train[:,i] for i in range(74)]
    model.fit(train_list, y_train, epochs=100, batch_size=6, verbose=0)
    # evaluate the model
    x_test = x[test]
    y_test = y[test]
    test_list = [x_test[:,i] for i in range(74)]
    scores = model.evaluate(test_list, y_test, verbose=0)

    print("%.2f%%" % (scores[1] * 100)) # Accuracy
    all_acc.append(scores[1] * 100)

# print("all acc: {}".format(all_acc))
print("mean acc: {}".format(np.mean(all_acc)))
print("std acc: {}".format(np.std(all_acc)))
acc_list.append(np.mean(all_acc))
std_list.append(np.std(all_acc))
all_acc = []
end = time.time()
print("%.2f" % (end - start))   # run time



In [8]:
#@title MODMA 128通道 简易弄一下
from sklearn.model_selection import StratifiedKFold
from keras.utils import to_categorical
import numpy as np
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from keras.layers import Input, Conv2D, MaxPooling2D, Dropout,Bidirectional
from keras.layers import Flatten, Dense, Concatenate, Reshape, LSTM
from keras.models import Sequential, Model

import keras
from keras import backend as K
import time
from sklearn.model_selection import StratifiedKFold


num_classes = 2
batch_size = 60
# eegnet_x,eegnet_y均为128通道静息
# eegnet_x_3ch等为3通道
x = np.load('/content/drive/MyDrive/new_MODMA_CRNN_128.npy')
y = np.load('/content/drive/MyDrive/EEGNet_Y.npy')
x = x.reshape(318,50,16,19,5)
# x = x.transpose((0,1,4,2,3))
y = np.repeat(y,6)
y = to_categorical(y, num_classes)
# print(x_alpha.shape)
print(x.shape)
print(y.shape)

acc_list = []
std_list = []
all_acc = []

start = time.time()
# x_train, x_test, y_train, y_test = train_test_split(one_falx, one_y, test_size=0.25)
seed = 7
np.random.seed(seed)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
cvscores = []

# create model
for train, test in kfold.split(x, y.argmax(1)):
    img_size = (16, 19, 5)

    # model.summary()
    def create_base_network(input_dim):
      seq = Sequential()
      seq.add(Conv2D(64, 5, activation='relu', padding='same', name='conv1', input_shape=input_dim))
      seq.add(Conv2D(128, 4, activation='relu', padding='same', name='conv2'))
      seq.add(Conv2D(256, 4, activation='relu', padding='same', name='conv3'))
      seq.add(Conv2D(64, 3, activation='relu', padding='same', name='conv4'))
      seq.add(MaxPooling2D(5, 2, name='pool1'))
      seq.add(Flatten(name='fla1'))
      seq.add(Dense(512, activation='relu', name='dense1'))
      seq.add(Reshape((1, 512), name='reshape'))
      return seq
    base_network = create_base_network(img_size)
    input_list = []
    for i in range(50):
      m = Input(shape=img_size)
      input_list.append(m)
    base_list = []
    for i in range(50):
      m = base_network(input_list[i])
      base_list.append(m)
    out_all = Concatenate(axis=1)(base_list)
    newout = LSTM(288)(out_all)
    out_layer = Dense(2, activation='softmax', name='out')(newout)
    model = Model(inputs = input_list, outputs = out_layer)
    # Compile model
    adam = Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08,weight_decay = 0.01)
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=Adam(),
                  metrics=['accuracy'])
    # Fit the model
    x_train = x[train]
    y_train = y[train]
    train_list = [x_train[:,i] for i in range(50)]
    model.fit(train_list, y_train, epochs=100, batch_size=60, verbose=1)
    # evaluate the model
    x_test = x[test]
    y_test = y[test]
    test_list = [x_test[:,i] for i in range(50)]
    scores = model.evaluate(test_list, y_test, verbose=1)

    print("%.2f%%" % (scores[1] * 100)) # Accuracy
    all_acc.append(scores[1] * 100)

# print("all acc: {}".format(all_acc))
print("mean acc: {}".format(np.mean(all_acc)))
print("std acc: {}".format(np.std(all_acc)))
acc_list.append(np.mean(all_acc))
std_list.append(np.std(all_acc))
all_acc = []
end = time.time()
print("%.2f" % (end - start))   # run time



(318, 50, 16, 19, 5)
(318, 2)
Epoch 1/100
5/5 [==============================] - 15s 285ms/step - loss: 1.2982 - accuracy: 0.5472
Epoch 2/100
5/5 [==============================] - 1s 283ms/step - loss: 0.6918 - accuracy: 0.5394
Epoch 3/100
5/5 [==============================] - 1s 288ms/step - loss: 0.7181 - accuracy: 0.4528
Epoch 4/100
5/5 [==============================] - 1s 286ms/step - loss: 0.7127 - accuracy: 0.4528
Epoch 5/100
5/5 [==============================] - 2s 291ms/step - loss: 0.6923 - accuracy: 0.5000
Epoch 6/100
5/5 [==============================] - 1s 295ms/step - loss: 0.6881 - accuracy: 0.5472
Epoch 7/100
5/5 [==============================] - 2s 358ms/step - loss: 0.6916 - accuracy: 0.5472
Epoch 8/100
5/5 [==============================] - 2s 307ms/step - loss: 0.6914 - accuracy: 0.5472
Epoch 9/100
5/5 [==============================] - 2s 296ms/step - loss: 0.7012 - accuracy: 0.4528
Epoch 10/100
5/5 [==============================] - 2s 295ms/step - loss: 0.69

2/2 [==============================] - 3s 80ms/step - loss: 0.6887 - accuracy: 0.5469
54.69%
Epoch 1/100
5/5 [==============================] - 13s 270ms/step - loss: 1.4448 - accuracy: 0.4606
Epoch 2/100
5/5 [==============================] - 1s 267ms/step - loss: 0.6889 - accuracy: 0.5354
Epoch 3/100
5/5 [==============================] - 1s 268ms/step - loss: 0.7192 - accuracy: 0.4528
Epoch 4/100
5/5 [==============================] - 1s 266ms/step - loss: 0.6909 - accuracy: 0.5394
Epoch 5/100
5/5 [==============================] - 1s 266ms/step - loss: 0.6920 - accuracy: 0.5512
Epoch 6/100
5/5 [==============================] - 1s 267ms/step - loss: 0.7011 - accuracy: 0.4528
Epoch 7/100
5/5 [==============================] - 2s 302ms/step - loss: 0.6934 - accuracy: 0.5472
Epoch 8/100
5/5 [==============================] - 1s 277ms/step - loss: 0.7032 - accuracy: 0.5472
Epoch 9/100
5/5 [==============================] - 1s 277ms/step - loss: 0.6943 - accuracy: 0.5394
Epoch 10/100
5/

In [ ]:
#@title SEED CNN代码备份留档
import tensorflow as tf
import numpy as np
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout,Bidirectional
from keras.layers import Flatten, Dense, Concatenate, Reshape, LSTM
from keras.models import Sequential, Model

import keras
from keras import backend as K
import time
from sklearn.model_selection import StratifiedKFold


num_classes = 3
batch_size = 128
img_rows, img_cols, num_chan = 8, 9, 4

img_size = (img_rows, img_cols, num_chan)


falx = np.load('/content/drive/MyDrive/t4x_89.npy')
y = np.load('/content/drive/MyDrive/t4y_89.npy')
# one_y = np.array([y[:1697]] * 3).reshape((-1,))
# one_y = to_categorical(one_y, num_classes)
one_y_1 = np.array([y[:1692]] * 3).reshape((-1,))
one_y_1 = to_categorical(one_y_1, num_classes)

acc_list = []
std_list = []
all_acc = []
for nb in range(1):
    K.clear_session()
    start = time.time()
    one_falx_1 = falx[nb * 3:nb * 3 + 3]
    one_falx_1 = one_falx_1.reshape((-1, 4, img_rows, img_cols, 5))

    one_y = one_y_1
    one_falx = one_falx_1[:,:,:,:,1:5]
    one_falx = one_falx.transpose((0,2,3,1,4))
    one_falx = one_falx.reshape(-1,8,9,16)
    print(one_y.shape)
    print(one_falx.shape)
    # x_train, x_test, y_train, y_test = train_test_split(one_falx, one_y, test_size=0.25)
    seed = 7
    np.random.seed(seed)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    cvscores = []

    # create model
    for train, test in kfold.split(one_falx, one_y.argmax(1)):
        img_size = (img_rows, img_cols, num_chan*4)

        # model.summary()
        def create_base_network(input_dim):
          seq = Sequential()
          seq.add(Conv2D(64, 5, activation='relu', padding='same', name='conv1', input_shape=input_dim))
          seq.add(Conv2D(128, 4, activation='relu', padding='same', name='conv2'))
          seq.add(Conv2D(256, 4, activation='relu', padding='same', name='conv3'))
          seq.add(Conv2D(64, 1, activation='relu', padding='same', name='conv4'))
          seq.add(MaxPooling2D(2, 2, name='pool1'))
          seq.add(Flatten(name='fla1'))
          seq.add(Dense(512, activation='relu', name='dense1'))
          return seq
        base_network = create_base_network(img_size)
        
        input_4 = Input(shape=img_size)
        out_all = base_network(input_4)
        out_layer = Dense(3, activation='softmax', name='out')(out_all)
        model = Model(inputs = input_4, outputs = out_layer)
        # lstm = Bidirectional(LSTM(128))(out_all)
        # # tcn_layer = tcn_model((512, 4),3)(out_all)
        # # fat = Flatten(tcn_layer)
        # out_layer = Dense(3, activation='softmax', name='out')(lstm)
        # model = Model(inputs = [input_1, input_2, input_3, input_4], outputs = out_layer)
        # Compile model
        model.compile(loss=keras.losses.categorical_crossentropy,
                      optimizer=keras.optimizers.Adam(),
                      metrics=['accuracy'])
        # Fit the model

        x_train = one_falx[train]
        print('x train ',x_train.shape)
        y_train = one_y[train]

        model.fit(x_train, y_train, epochs=100, batch_size=128, verbose=0)
        # evaluate the model
        x_test = one_falx[test]
        y_test = one_y[test]
        scores = model.evaluate(x_test, y_test, verbose=0)

        print("%.2f%%" % (scores[1] * 100)) # Accuracy
        all_acc.append(scores[1] * 100)

    # print("all acc: {}".format(all_acc))
    print("mean acc: {}".format(np.mean(all_acc)))
    print("std acc: {}".format(np.std(all_acc)))
    acc_list.append(np.mean(all_acc))
    std_list.append(np.std(all_acc))
    print("进度： {}".format(nb))
    all_acc = []
    end = time.time()
    print("%.2f" % (end - start))   # run time
print('Acc_all: {}'.format(acc_list))
print('Std_all: {}'.format(std_list))
print("Acc_mean: {}".format(np.mean(acc_list)))
print("Std_all: {}".format(np.std(std_list)))


(5076, 3)
(5076, 8, 9, 16)


TypeError: ignored